In [0]:
import pandas as pd
import plotly.express as px

# Read data from Spark SQL
df = spark.sql("select * from classes.gold.df_cirurgias").toPandas()

# Check column names
print(df.columns)

# Use the correct column name for 'resultado'
# For example, if the column is 'resultado_cirurgia', update accordingly:
df['data_cirurgia'] = pd.to_datetime(df['data_cirurgia'], errors='coerce')

sunburst_data = (
    df.groupby(['tipo_cirurgia', 'doenca', 'resultado_cirurgia'])
    .size()
    .reset_index(name='total')
    .sort_values('total', ascending=False)
)

fig = px.sunburst(
    sunburst_data,
    path=['tipo_cirurgia', 'doenca', 'resultado_cirurgia'],
    values='total',
    color='total',
    color_continuous_scale='RdYlGn',
    title="<b>Distribuição Hierárquica de Cirurgias no SUS (2020–2025)</b><br><sup>Clique para expandir • Hover para detalhes</sup>",
    height=800,
    maxdepth=3
)

fig.update_layout(
    title_x=0.5,
    font=dict(size=14, family="Arial"),
    coloraxis_colorbar=dict(
        title="Total de Cirurgias",
        tickformat=',',
        len=0.5,
        y=0.5
    ),
    margin=dict(t=100, l=0, r=0, b=0)
)

fig.update_traces(
    hovertemplate="<b>%{label}</b><br>Total: %{value:,.0f}<br>Participação: %{percentParent:.1%} do pai<extra></extra>"
)

fig.show()